In [8]:
import os 
import keras
from keras.models import Sequential 
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout, BatchNormalization
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
plt.style.use('dark_background')
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

One Hot Encoding The Target Classes

In [9]:
encoder = OneHotEncoder()
encoder.fit([[0], [1]])
# 0 = Tumor
# 1 = Normal

OneHotEncoder()

Creating 3 Important Lists _

1. data list for storing image data in numpy array form
2. paths list for storing paths of all images
3. result list for storing one hot encoded form ot target class whether normal or tumor

In [10]:
data = []
paths = []
results = []


for r, d, f in os.walk(r'../input/brain-mri-images-for-brain-tumor-detection/yes'):
    for file in f:
        if '.jpg' in file:
            paths.append(os.path.join(r, file))
for path in paths:
    img = Image.open(path)
    img = img.resize((128, 128))
    img = np.array(img)
    if(img.shape == (128, 128)):
        data.append(np.array(img))
        result.append(encoder.transform([[0]]).toarray())
